# BE noté -- Fondations statistiques du Machine Learning

Cet devoir noté est composé de deux exercices. Il sera idéalement réalisé en binome et éventuellement seul. Les réponses seront données dans un notebook qui indiquera clairement les **noms et prénoms des élèves** l'ayant realisé.


## Exercice 1


Afin d'estimer efficacement le niveau de fatigue des ailes d'un d'avion au cours des années, il a été proposé de lancer une étude pour évaluer s'il était possible de déduire le niveau de stress subit par les ailes de l'avion lors de phases de vols diverses avec de données capteurs acquises en routine pendant les vols. Une personne ayant une expertise mécanique sur le modèle d'avion étudié a alors quantifié le niveau de stress subi par les ailes dans différentes phases de vols et différents contextes. Nous allons mettre en lien ces niveaux de stress avec des données capteurs acquises au même moment que les annotations. Nous allons pour cela utiliser la régression linaire. 


### QUESTION 1.1

Les données d'apprentissage sont dans les fichiers *E1_sensor_vals.csv* et *E1_stress_vals.csv*. Ouvrez ces fichiers et mettez les données dans des numpy arrays ou des pandas dataframes *X* et *Y*. Représentez alors le lien entre les valeurs issues de chaque capteur et le niveau de stress dans des nuages de points 2D. Identifiez-vous des relations entre des données capteur et le niveau de stress ? Quels capteurs vous paraissent être les plus pertinents.

## **Réponse :**

Les *sensor_01*, *sensor_12* et *sensor_15* apparaissent comme les plus pertinents en termes de relations entre données capteur et niveau de stress : leur nuage de point semblent mettre en avant une relation linéaire.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy import stats
import statsmodels.api as sm

# Charger les données
X = pd.read_csv('E1_sensor_vals.csv', sep=';')
Y = pd.read_csv('E1_stress_vals.csv', sep=';')

# Nombre de capteurs
num_sensors = X.shape[1]

# Créer des nuages de points pour chaque capteur
for i in range(num_sensors):
    plt.figure(figsize=(10, 6))
    plt.scatter(X.iloc[:, i], Y, alpha=0.5)
    plt.title(f'Relation entre le capteur {i+1} et le niveau de stress')
    plt.xlabel(f'Valeurs du capteur {i+1}')
    plt.ylabel('Niveau de stress')
    plt.grid()
    plt.show()


### QUESTION 1.2
On se demande s'il est possible de prédire le niveau de stress à partir d'**une seule** des variables *sensor_01*, *sensor_12* ou *sensor_15*.


#### QUESTION 1.2.1

Effectuez une régression linéaire simple entre chacune de ces trois variables et le niveau de stress. Quelle stratégie de validation croisée vous semble être la plus adaptée dans ce contexte ?

## **Réponse :**

La stratégie des K-folds semble la plus pertinente dans notre contexte : elle est d'abord moins coûteuse en temps de calcul que la LOOCV, puis la taille des sous-échantillons est suffisamment faible pour éviter le surapprentissage.


#### QUESTION 1.2.2

Evaluez alors la qualité des prédictions en quantifiant les erreurs de prédiction au carré. Quelle variable vous semble être la plus pertinente pour prédire le niveau de stress et pourquoi ?

## **Réponse :**

La $MSE$ (resp. le $R^2$) est minimale (resp. maximal) pour le *sensor_15*, ce qui laisse à penser que c'est la variable la plus pertinente pour prédire le niveau de stress.


#### QUESTION 1.2.3

Peut-on statistiquement affirmer qu'il existe une relation significative entre le niveau de stress et (indépendament) *sensor_01*, *sensor_12* ou bien *sensor_15* ? Si oui, décrivez votre procédure de test.

## **Réponse :**

Oui, en mesurant la p-value d'un test d'hypothèse H0 "*Il n'existe pas de relation significative entre le niveau de stress et le capteur.*" et d'hypothèse H1 "*Il y a une relation significative entre ...*" pour chacun des capteurs. Si celle-ci est inférieure à 0.05, alors on pourra rejeter H0 et ainsi affirmer qu'il y a bel et bien une relation significative entre le niveau de stress et le capteur en question. Pour cela, on ajuste le modèle de régression linéaire via OLS du module statsmodel et l'on en tire la p-value.


In [ ]:
# Question 1.2.2

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

# Sélectionner les capteurs 1, 12 et 15 (index 0, 11, 14 en Python)
capteurs_indices = [0, 11, 14]  # Index des capteurs dans X

for i in capteurs_indices:

    # Sélectionner le capteur i
    X_i = X.iloc[:, i].values.reshape(-1, 1)  # Reshape pour avoir la bonne forme (n_samples, n_features)
    
    # Créer le modèle de régression linéaire
    model = LinearRegression()
    
    # Effectuer une validation croisée (discutée ci-dessous)
    scores = cross_val_score(model, X_i, Y, cv=5, scoring='neg_mean_squared_error')  # Validation croisée
    mse_scores = -scores  # Inverser les scores pour obtenir les MSE
    mean_mse = np.mean(mse_scores)  # Moyenne des MSE
    
    # Ajuster le modèle et faire des prédictions
    model.fit(X_i, Y)
    predictions = model.predict(X_i)
    
    # Calculer les métriques de performance
    r2 = r2_score(Y, predictions)
    rmse = np.sqrt(mean_squared_error(Y, predictions))
    
    # Affichage des résultats
    print(f"Capteur {i + 1}:")
    print(f"  MSE : {mean_mse:.2f}")
    print(f"  R²: {r2:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    
    # Tracer les résultats
    plt.figure(figsize=(10, 5))
    plt.scatter(X_i, Y, alpha=0.5, label='Données réelles')
    plt.plot(X_i, predictions, color='red', label='Régression linéaire', linewidth=2)
    plt.title(f"Capteur {i + 1} vs stress")
    plt.xlabel(f"Capteur {i + 1} - valeurs")
    plt.ylabel("Niveau de stress")
    plt.legend()
    plt.grid(True)
    plt.show()


### QUESTION 1.3

On s'intéresse maintenant au lien entre la variable *sensor_12* et le niveau de stress. On peut remarquer qu'il semble exister une relation linéaire entre ces variables, mais que les données contiennent aussi deux valeurs aberrantes.


#### QUESTION 1.3.1

**Stratégie 1** : Définissez une première procédure pour détecter automatiquement les deux données aberrantes dans un jeu de données. On utilisera pour cela les distances de Cook.

## **Réponse :**

Voici notre procédure :
- Nous ajustons un modèle de régression linéaire en utilisant la méthode des moindres carrés (OLS) pour modéliser la relation entre *sensor_12* et les niveaux de stress ;
- Nous calculons les **distances de Cook**, qui renvoie les distances de Cook pour chaque observation dans le jeu de données ;
- Nous définissons un **seuil** pour identifier les données aberrantes, en utilisant un seuil de **1**, une valeur couramment acceptée dans la littérature ;
- Nous extrayons les indices des observations ayant des distances de Cook supérieures à ce seuil, indiquant qu'elles sont considérées comme des **points aberrants** ;
- Nous créons un graphique en tige (stem plot) pour visualiser les distances de Cook de chaque observation, avec une ligne horizontale indiquant le seuil, afin de faciliter l'identification des observations aberrantes ;
- Nous affichons les indices et les valeurs des observations identifiées comme aberrantes.


#### QUESTION 1.3.2

**Stratégie 2** : Nous allons ici utiliser toutes les observations pour l'apprentissage du modèle linéaire et sa validation. En supposant que les erreurs de prédiction suivent une loi normale centrée, pourrait-on aussi détecter les outliers à partir d'un test d'hypothèse. Si oui, décrivez la procédure.

## **Réponse :**

Voici notre procédure :
- nous utilisons un **test de Shapiro-Wilk** avec la règle des 2-sigmas ;
- nous calculons d'abord **les résidus**, puis effectuons le test de normalité avec Shapiro-Wilk qui permet de valider ou non l'hypothèse que les résidus suivent une distribution normale avec la p-valeur ;
- nous calculons **la moyenne et l'écart-type des résidus** afin de définir les limites dans la règle des 2-sigmas ;
Une fois les seuils définis par la règle des 2-sigmas, nous observons les valeurs en dehors de ces limites : il s'agit des outliers ; 
- nous affichons ensuite ces résultats.

In [ ]:
from scipy import stats
import statsmodels.api as sm

# Question 1.3.1

# Sélectionner sensor_12
X_sensor_12 = sm.add_constant(X[['sensor_12']])  # Ajout d'une constante pour l'ordonnée à l'origine
model_sensor_12 = sm.OLS(Y, X_sensor_12).fit()  # Ajustement du modèle

# Calculer les distances de Cook
influence = model_sensor_12.get_influence()
cooks_d = influence.cooks_distance[0]

# Identifier les points aberrants basés sur un seuil
threshold = 1
outliers_indices = np.where(cooks_d > threshold)[0] # Indices des outliers

# Affichage des distances de Cook
plt.figure(figsize=(10, 6))
plt.stem(np.arange(len(cooks_d)), cooks_d, markerfmt=",", basefmt=" ")
plt.axhline(y=threshold, color='r', linestyle='--', label='Seuil')
plt.title("Distances de Cook")
plt.xlabel("Indices des observations")
plt.ylabel("Distance de Cook")
plt.legend()
plt.grid()
plt.show()

# Afficher les indices des valeurs aberrantes
outliers_sensor_12 = X.loc[outliers_indices, ['sensor_12']]
print("Indices des valeurs aberrantes (sensor_12):", outliers_indices)
print("Valeurs aberrantes :")
print(outliers_sensor_12)


# Question 1.3.2

# Calculer les résidus du modèle sensor_12
residuals = model_sensor_12.resid

# Effectuer le test de Shapiro-Wilk pour vérifier la normalité des résidus
shapiro_stat, p_value = stats.shapiro(residuals)
alpha = 0.05  # Seuil de signification
if p_value < alpha:
    print("Les résidus ne suivent pas une distribution normale (p-value < 0.05)")
else:
    print("Les résidus suivent une distribution normale (p-value >= 0.05)")

# Calculer la moyenne et l'écart type des résidus
mean_residual = np.mean(residuals)
std_residual = np.std(residuals)

# Définir les limites pour la règle des trois sigmas
lower_limit = mean_residual - 2 * std_residual
upper_limit = mean_residual + 2 * std_residual

# Identifier les outliers basés sur la règle des trois sigmas
outliers_indices = np.where((residuals < lower_limit) | (residuals > upper_limit))[0]

# Afficher les résultats
outliers_sensor_12 = X.loc[outliers_indices, ['sensor_12']]
print("Indices des valeurs aberrantes (sensor_12):", outliers_indices)
print("Valeurs aberrantes :")
print(outliers_sensor_12)

# Visualisation des résidus et des limites
plt.figure(figsize=(10, 6))
plt.plot(residuals, 'o', label='Résidus')
plt.axhline(y=lower_limit, color='red', linestyle='--', label='Limite inférieure (2 sigmas)')
plt.axhline(y=upper_limit, color='green', linestyle='--', label='Limite supérieure (2 sigmas)')
plt.title('Résidus du modèle sensor_12')
plt.xlabel('Indices des observations')
plt.ylabel('Résidus')
plt.legend()
plt.grid()
plt.show()


### QUESTION 1.4


Nous supprimerons dans cette question les deux observations qui sont aberrantes sur la variable *sensor_12*.

Nous souhaitons maintenant sélectionner automatiquement un nombre réduit, mais supérieur à 1, de capteurs qui nous permettraient de prédire au mieux le niveau de stress. Nous allons pour cela utiliser la régression multiple avec un terme de régularisation.

#### QUESTION 1.4.1

Quel traitement préalable allez-vous effectuer sur les données capteur et pourquoi ?

## **Réponse :**
 
Avant d'appliquer la régression multiple avec un terme de régularisation, il est important de réaliser certains traitements préalables sur les données des capteurs :
1. **Suppression des valeurs aberrantes**, qui peuvent fausser l'ajustement du modèle et affecter la qualité des prédictions.
2. **Vérification de la présence de valeurs manquantes dans les données**. Selon la quantité de données manquantes, vous pouvez choisir de les supprimer ou de les imputer avec des méthodes appropriées (comme la moyenne, la médiane ou des techniques plus avancées comme l'imputation par régression).
3. **Normalisation des données**, pour replacer toutes les données à la même échelle.

#### QUESTION 1.4.2

Décrivez votre démarche de sélection de variables et vos résultats. Est-ce que l'utilisation des données capteurs vous semble fiable ?

## **Réponse :**

Voici notre démarche de sélection de variables :

1. Après avoir supprimé les valeurs aberrantes de la variable *sensor_12*, on vérifie et gère les valeurs manquantes et normalise les données des capteurs.
2. On effectue une analyse de corrélation entre les capteurs et le niveau de stress pour identifier les capteurs qui montrent une relation significative. Cela peut être fait en utilisant des matrices de corrélation.
3. On utilise des méthodes de régression avec régularisation (ici Lasso) pour sélectionner les variables les plus pertinentes. Ces méthodes aident à réduire le surajustement en pénalisant les coefficients des variables non pertinentes.
4. On met en œuvre une validation croisée pour évaluer la performance du modèle et s'assurer que la sélection des variables n'était pas due au hasard. Cela aide à évaluer la robustesse des variables sélectionnées.
5. On mesure la performance du modèle à l'aide de la $MSE$ et du $R^2$ pour déterminer la qualité des prédictions.

### **Analyse des résultats :**

La régression Lasso par validation croisée sur les données données d'entrainement sélectionnent 2 capteurs (*sensor_12* et *sensor_15*) pour prédire les niveaux de stress (ie dont les coefficients $β$ sont non nuls). Ensuite, le graphe représentant les coefficients $β$ de la régression Lasso sur l'ensemble des données fait ressortir 4 capteurs au total (dont *sensor_12* et *sensor_15* pour lesquels les $β$ sont significativement plus grands). La différence dans le nombre de capteurs mis en avant s'explique ainsi : lors de l'entraînement initial avec validation croisée, différentes valeurs de régularisation $α$ sont testées pour trouver celle qui minimise l'erreur. Ensuite, lors du fit final sur l'ensemble complet des données, la valeur optimale de $α$ est utilisée. Cependant, les coefficients peuvent légèrement changer, et même si seulement deux capteurs étaient sélectionnés initialement, d'autres capteurs peuvent obtenir des coefficients non nuls lors du réajustement sur toutes les données. Cela est dû à l'augmentation de la quantité de données utilisée.

### **Conclusion :**

Au des coefficients de Lasso, nous estimons que l'utilisation de *sensor_12* et *sensor_15* est pertinente pour prédire les niveaux de stress sur l'aile d'avion.


#### QUESTION 1.4.3

Quelle démarche auriez-vous pour rendre compte des résultats de cette étude sachant que les données contenaient tout de même deux outliers ?

## **Réponse :**

Malgré la présence d'outliers, nous pouvons interpréter les résultats en précisant que deux outliers ont été détectés et supprimés pour améliorer les prédictions faites par le modèle. Ici, nous avons même établi deux stratégies pour valider l'identification des outliers. 
Nous pouvons comparer les performances des modèles avec et sans les outliers (en regardant la MSE et R²). 

Enfin, nous aurions pu utiliser la régression par moindres carrés pondérés à la place des moindres carrés ordinaires qui est moins sensible aux outliers. La régression est plus robuste car des poids plus faibles sont appliqués aux valeurs aberrantes qui ont été déterminées par une analyse préliminaire des résidus du modèles OLS. Ainsi, l'utilisation de ces poids réduisent l'impact des valeurs aberrantes sur les estimations des paramètres.

In [ ]:
# Question 1.4.2

import seaborn as sns
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Charger les données des capteurs et des niveaux de stress
data = pd.read_csv('E1_sensor_vals.csv', sep=';')
Y = pd.read_csv('E1_stress_vals.csv', sep=';')

# Suppression des valeurs aberrantes
aberrant_indexes = [2, 62]  # Indices identifiés précédemment
data = data.drop(index=aberrant_indexes)
Y = Y.drop(index=aberrant_indexes)

# Gestion des valeurs manquantes
data = data.dropna()  # Suppression des lignes avec valeurs manquantes

# Normalisation des données (sur les capteurs)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data.drop(columns=data.columns[-1]))  # Ne pas inclure la colonne Stress

# Séparation des données
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Régression Lasso avec validation croisée
lasso = LassoCV(cv=5)  # Utilisation de 5-fold cross-validation
lasso.fit(X_train, Y_train)

# Sélection des variables
selected_features = np.where(lasso.coef_ != 0)[0]
print(f"Capteurs sélectionnés : {data.columns[:-1][selected_features].tolist()}")

# Évaluation du modèle
mse = cross_val_score(lasso, X_train, Y_train, scoring='neg_mean_squared_error', cv=5)
print(f"MSE (validation croisée) : {-np.mean(mse):.3f}")

# Ajustement final sur l'ensemble d'apprentissage
lasso.fit(X_scaled, Y)

# Performance sur l'ensemble de test
test_mse = mean_squared_error(Y_test, lasso.predict(X_test))
print(f"MSE sur l'ensemble de test : {test_mse:.3f}")

# Affichage des coefficients du modèle
coefficients = pd.Series(lasso.coef_, index=data.columns[:-1])
plt.figure(figsize=(10, 6))
coefficients.plot(kind='bar')
plt.title('Coefficients de la régression Lasso')
plt.xlabel('Capteurs')
plt.ylabel('Coefficient')
plt.show()

In [50]:
# Question 1.4.3


## Exercice 2

Nous souhaitons évaluer si un nouveau produit a un effet significatif sur le rendement de moteurs. Pour y répondre, ce rendement (*Efficiency*) a été mesuré sur deux types de moteurs (*Brand_1* et *Brand_2*) et en testant différents niveaux de concentration (*Concentration*) du produit. Les observations sont dans le fichier *E2_Efficiency_Obs.csv*. A l'aide de modèles de régression linéaire et de tests statistiques, nous allons alors évaluer :
- Le produit semble-t-il avoir un effet ?
- Cet effet est-il différent en fonction de la marque du moteur ?
- Cet effet dépend-il de la concentration du produit ?

Afin de résoudre le problème, deux hypothèses seront effectuées :
- Pour chaque marque de moteur, la relation entre la concentration et le rendement est supposée linéaire.
- La distribution des erreurs de ce modèle est supposée suivre une loi Normale centrée.


La démarche pour répondre à ces questions sera commentée dans le notebook rendu, quels qu'en soient ses résultats.


Conseil : Avant de définir une stratégie de résolution, il est recommandé de visualiser les données de *E2_Efficiency_Obs.csv* en distinguant bien les observations obtenues dans les groupes *Brand_1* et *Brand_2*.


## **Réponse :**

### 1. **Le produit semble-t-il avoir un effet ?**

A l'affichage des données, il semble en effet y avoir un effet significatif sur le rendement des moteurs. Après régression linéaire effectuée entre le produit et *Brand_1* et *Brand_2*, on obtient les informations suivantes :
- Pour *Brand_1*, la concentration du produit a un effet hautement significatif sur l'efficacité, avec un coefficient de 0.4915 et une p-valeur proche de 0 (p < 0.001). Cela montre qu'une augmentation de la concentration entraîne une augmentation significative du rendement ;
- Pour *Brand_2*, le coefficient de la concentration est de **0.3090** avec une p-valeur de **0.000**. Cela confirme également que le produit a un effet significatif sur le rendement des moteurs de cette marque. En effet, une augmentation de la concentration de 1 unité est associée à une augmentation de l'efficacité de **0.3090**.

### 2. **Cet effet est-il différent en fonction de la marque du moteur ?**

Oui, l'effet du produit semble dépendre de la marque du moteur.
- Le modèle avec interaction entre la **concentration** et la **marque** permet d'évaluer si cet effet est différent selon les marques. Si le terme d'interaction (`Concentration:Brand_binary`) est significatif, cela indiquerait que l'effet de la concentration sur l'efficacité est différent pour *Brand_1* et *Brand_2*. Les résultats du modèle d'interaction (résumé à compléter) devraient montrer si cet effet est statistiquement différent entre les deux marques. Il est pertinent de comparer les coefficients des deux marques : *Brand_1* (0.4915) semble avoir un effet plus fort que *Brand_2* (0.3090), ce qui pourrait suggérer des différences dans la manière dont chaque marque réagit à la concentration du produit.

### 3. **Cet effet dépend-il de la concentration du produit ?**

Oui, l'effet du produit dépend directement de la concentration, comme montré par les régressions.
- Pour *Brand_1*, la relation entre la concentration et l'efficacité est linéaire et positive, ce qui signifie que plus la concentration du produit augmente, plus le rendement du moteur augmente de manière prévisible.
- Pour *Brand_2*, la relation est également positive, mais avec un coefficient inférieur. Cela pourrait indiquer que bien que l'efficacité augmente avec la concentration, l'ampleur de l'effet est plus faible comparée à *Brand_1*. Si le terme d'interaction est significatif, cela pourrait également suggérer que la dépendance à la concentration est différente pour *Brand_2*, indiquant une interaction entre la concentration et la marque.

### Conclusion :
Le produit a un effet significatif sur l'efficacité des moteurs, comme en témoigne les coefficients positifs pour les deux marques. Cependant, cet effet varie en fonction de la marque, avec une influence plus forte observée pour *Brand_1*. Cela justifie un ajustement des stratégies d'utilisation du produit en fonction du type de moteur, en tenant compte que *Brand_2* pourrait bénéficier davantage d'augmentations de concentration.


In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols

# Charger les données depuis le fichier CSV
data = pd.read_csv('E2_Efficiency_Obs.csv')

# Visualisation des données : Nuages de points pour chaque marque
sns.lmplot(x='Concentration', y='Efficiency', hue='Brand', data=data, aspect=1.5, ci=None)
plt.title('Relation entre la concentration et l\'efficacité selon la marque')
plt.show()

# Séparer les données par marque
brand_1_data = data[data['Brand'] == 'Brand_1']
brand_2_data = data[data['Brand'] == 'Brand_2']

"""
# Ajuster le modèle pour Brand_1
model_brand_1 = ols('Efficiency ~ Concentration', data=brand_1_data).fit()
print("Régression pour Brand_1 :")
print(model_brand_1.summary())
"""

# Ajuster le modèle pour Brand_2
model_brand_2 = ols('Efficiency ~ Concentration', data=brand_2_data).fit()
print("\nRégression pour Brand_2 :")
print(model_brand_2.summary())

# Ajouter une variable binaire pour la marque (Brand_2 = 1, Brand_1 = 0)
data['Brand_binary'] = data['Brand'].apply(lambda x: 1 if x == 'Brand_2' else 0)

# Ajuster le modèle avec interaction entre marque et concentration
interaction_model = ols('Efficiency ~ Concentration * Brand_binary', data=data).fit()

# Résumé du modèle avec interaction
print("\nModèle avec interaction entre la concentration et la marque :")
print(interaction_model.summary())
